# Colombia dataset

  * Author: Esteban Cid y Jesús Cid
  * Version 0.0: Jan, 2024.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score, davies_bouldin_score

try:
    from pyclustering.cluster.kmedians import kmedians
    from cartopy.io import shapereader
    from ortools.constraint_solver import pywrapcp
    from ortools.constraint_solver import routing_enums_pb2
except:
    !pip install pyclustering
    !pip install cartopy
    !pip install ortools
    from pyclustering.cluster.kmedians import kmedians
    from cartopy.io import shapereader
    from ortools.constraint_solver import pywrapcp
    from ortools.constraint_solver import routing_enums_pb2

import geopandas
from shapely.geometry import mapping

from geopy.distance import geodesic

from time import time
import os
import copy

In [ ]:
# Para leer el fichero de datos directamente de google drive
try:
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir("/content/drive/My Drive/TFM_Esteban")
    in_colab = True
except:
    in_colab = False

In [ ]:
# ######################################
# Parámetros de configuracion de usuario
# ######################################

# Numero de centros de distribución.
# Poner aqui un valor de prueba. El valor definitivo debe elegirse después de
# la primera ejecución, en función de los resultados de las metricas de cluster
# (silouhette, davis-boulding, etc)
num_centros = 7

# Parámetros del CVRP ---------------------------------------------------------

# Nivel de demanda
demanda_media = 2

# Capacidad de cada vehiculo (mejor no modificarlo)
vc = 3_500

# Periodo de demanda (escribir 'all' para usar el acumulado de todos los meses)
mes = '202207'

# Tiempo máximo de optimización, por cada cluster.
# Limita el tiempo de búsqueda de rutas al optimizador.
# Tiempos largos garantizan mejores soluciones. Tiempos cortos proporcionan
# soluciones subóptimas. Si es demasiado corto, el optimizador puede no
# encontrar ninguna solución
tmax = 500

# Cluster utilizado para hacer la prueba de planificacion de rutas con un solo
# centro de distribución
cluster_de_prueba = 5

# ###################
# Parámetros técnicos
# ###################

# Máx. nº de tiendas
# Poner valor pequeño para pruebas, y grande para usar el dataset completo
nmax = 999999999999

# Número de ciudades para el tsp. Poner un valor menor que el total de ciudades
# para hacer pruebas que no duren demasiado
# (este parámetro no se usa en la planificación de rutas mediante CVRP)
n_loc_tsp = 800

# Métrica de distancia
# A elegir entre "euclidea" (rápida pero imprecisa), "haversine" (algo más
# lenta pero bastante más precisa) o "geodesic" (muy precisa pero muy lenta)
distance_metric = "haversine"

# Escala para conversión a distancias enteras en ortools.
scale_D = 1e9
# Escala para conversión a demandas enteras en ortools.
scale_M = 1e6

# #######################
# Parámetros de ejecución
# #######################

# Estos parámetros sirven para filtrar secciones de código que no se deseen
# ejecutar
compute_silouhette = False
compute_distances = False
compute_kmedians = False
compute_tsp = False
compute_ckdb = False
remove_zero_demand_cities = True
compute_pruebavrp = False



## 1. Carga de datos.

The dataset contains coordinate of populations from Colombia. Coordinates are given by latitude and longitude. You should evaluate geodesic distances to evaluate the costs.

### 1.1. Lectura del fichero de datos

En primer lugar, cargamos los datos del fichero en una tabla (dataframe). Podemos previsualizar el contenido de las primeras entradas de la tabla.

A continuación

In [ ]:
# Load data
df_cities = pd.read_excel('Base_anonima_curada2.xlsx')
df_cities.head()

Los atributos de cada entrada son:

In [ ]:
print(df_cities.columns)

### 1.2. Extracción de coordenadas geograficas y pesos.

El atributo 'latitud longitud' contiene las coordenadas geográficas en formato texto.

In [ ]:
if mes == 'all':
   mes = 'Acumuladas (Calculado por mí)'

# Extrae datos de la columna 'latitud longitud'
geodata_str = df_cities['latitud longitud'].tolist()
# Extrae datos de demanda monetaria acumulada
demanda_str = df_cities['Acumuladas (Calculado por mi)'].tolist()
demandames_str = df_cities[mes].tolist()

# Hay que eliminar las entradas sin coordenadas
# (esto es necesario en el dataset original, pero quizás ya no hace falta con
# el dataset curado)

# Suprime entradas vacías
geodata_str, demanda_str, demandames_str = zip(
    *[(g, w, z) for g, w, z in zip(geodata_str, demanda_str, demandames_str) if g != 0])
geodata_str = list(geodata_str)
demanda_str = list(demanda_str)
demandames_str = list(demandames_str)

# Separa latitud de longitud
geodata_str = [x.split(',') for x in geodata_str]

# Corrige errores de formato en los datos, y convierte latitud y longitud a
# números en coma flotante
geodata_x = []
demanda_x = []
demandames_x=[]
for x, w, z in zip(geodata_str, demanda_str, demandames_str):
    if len(x) < 2:
        print(f'No hay comas: {x}')
    elif '------------' in x[1]:
        print(f'Lineas continuas: {x}')
    elif 's ' in x[0]:
        print(f'Entrada eliminada: {x}')
    elif x[1][-1] in {'.', 'v'}:
        print(f'Entrada eliminada: {x}')
    else:
        # Note that we take coordinates in reverse order
        geodata_x.append([float(x[1]), float(x[0])])
        demanda_x.append(w)
        demandames_x.append(z)

Algunas entradas tienen localizaciones incorrectas, fuera del mapa de Colombia, y deben suprimirse.

In [ ]:
# Filtrado por coordenadas
geodata_x2 = []
demanda_x2 = []
demandames_x2 = []
for x, w, z in zip(geodata_x, demanda_x, demandames_x):
    # Filtrado por longitud
    if x[1] < -10 or x[1] > 11.99:
        print(f'Entrada eliminada: {x}')
    # Filtrado por latitud
    elif x[0] < -80 or x[0] > -69.99:
        print(f'Entrada eliminada: {x}')
    else:
        geodata_x2.append([x[0], x[1]])
        demanda_x2.append(w)
        demandames_x2.append(z)

# Demanda, en unidades monetarias.
demanda_x2 = np.array(demanda_x2)
demandames_x2 = np.array(demandames_x2)
# Perfil de demanda normalizado (suma 1)
d_profile = demandames_x2 / np.sum(demandames_x2)

Los datos ya están preparados, y pueden cargarse en una matrix, ${\bf X}$.

In [ ]:
# Load GPS coordinates in a numpy array
X = np.array(geodata_x2)
n_cities = X.shape[0]

print("-- Bounding box")
print(f"-- -- Longitud: Mínima: {np.min(X[:, 0])}, máxima: {np.max(X[:, 0])}")
print(f"-- -- Latitud: Mínima: {np.min(X[:, 1])}, máxima: {np.max(X[:, 1])}")
print(f'Dataset cargado con {n_cities} ciudades')

Tranformaremos la matriz de datos para que las unidades sean aproximadamente kilométricas:

In [ ]:
def flat_factors(verbose=False):
    """
    De termina los factores de escalado que deben aplicarse a las coordenadas
    geodesicas para aproximar el resultado por un plano
    """

    # Definimos en primer lugar un espacio delimitado por dos paralelos y dos
    # meridianos sobre la superficie terrestre, que aproximaremos por un
    # rectángulo en el plano.
    lat_min, lat_max = 0.16073, 11.71425
    long_min, long_max = -78.7934, -70.74338

    # Estos son los vertices del espacio definido por los meridianos y paralelos
    box = np.array([[lat_min, long_min],
                    [lat_min, long_max],
                    [lat_max, long_min],
                    [lat_max, long_max]])

    # Para trasformar las coordenadas geográficas en kilométricas, calculamos
    # primero las distancias geodésicas entre estos vértices, en kilómetros
    distWE_por_el_N_total = geodesic(box[0], box[1]).kilometers
    distWE_por_el_S_total = geodesic(box[2], box[3]).kilometers
    dist_NS1_total = geodesic(box[0], box[2]).kilometers
    dist_NS2_total = geodesic(box[1], box[3]).kilometers

    # Distancias W-E en unidaddes de "incrementos de longitud"
    distWE_por_el_N = distWE_por_el_N_total / (long_max - long_min)
    distWE_por_el_S = distWE_por_el_S_total / (long_max - long_min)
    # Distancias N-S en unidaddes de "incrementos de latitud"
    dist_NS1 = dist_NS1_total / (lat_max - lat_min)
    dist_NS2 = dist_NS2_total / (lat_max - lat_min)

    if verbose:
        print("Distancias totales:")
        print(distWE_por_el_N_total)
        print(distWE_por_el_S_total)
        print(dist_NS1_total)
        print(dist_NS2_total)

        print("Distancias por unidad de coordenada geográfica:")
        print(distWE_por_el_N)
        print(distWE_por_el_S)
        print(dist_NS1)
        print(dist_NS2)

    # Factores que habrá que aplicar a los incrementos N-S y W-E para convertirlos
    # en distancias kilometricas aproximadas
    factor_NS = dist_NS1;
    factor_WE = (distWE_por_el_N + distWE_por_el_S) / 2;

    return factor_NS, factor_WE

def flat_projection(X, verbose=False):
    """
    Proyecta las coordenadas geodesicas en X, de forma aproximada, sobre un
    plano

    Parameters
    ----------
    X : numpy array
        Matriz de coordenadas geodesicas
    """

    factor_NS, factor_WE = flat_factors(verbose=verbose)

    # Matriz Z, del mismo tamaño que X, pero con ceros
    Z = np.zeros(X.shape);

    # Rellena con las coordenadas proyectadas
    Z[:, 1] = factor_NS * X[:, 1]
    Z[:, 0] = factor_WE * X[:, 0]

    return Z

def inverse_flat_projection(Z):
    """
    Recupera las coordenadas geodesicas a partir de las coordenadas proyectadas
    en Z

    Parameters
    ----------
    X : numpy array
        Matriz de coordenadas geodesicas
    """

    factor_NS, factor_WE = flat_factors(verbose=False)

    # Matriz Z, del mismo tamaño que X, pero con ceros
    X = np.zeros(Z.shape);

    # Rellena con las coordenadas proyectadas
    X[:, 1] = Z[:, 1] / factor_NS
    X[:, 0] = Z[:, 0] / factor_WE

    return X

Z = flat_projection(X, verbose=True)

## 2. Visualización de datos.

A continuación cargaremos las coordenadas geográficas del contorno de colombia, para visualizar los datos.

In [ ]:
# Lee datos de las fronteras de Colombia
shpfilename = shapereader.natural_earth('50m', 'cultural', 'admin_0_countries')
dfC = geopandas.read_file(shpfilename)
Cdata = dfC.loc[dfC['SOVEREIGNT']=='Colombia', 'geometry'].values[0]

# Extrae los puntos que definen la fromtera de Colombia, definidos sobre el
# rectangulo que hemos definido como espacio de trabajo
C = mapping(Cdata)['coordinates']
cx, cy = zip(*C[0][0])
# Frontera en coordenadas geodesicas
Cx = np.array([cx, cy]).T
# Frontera en coordenadas proyectadas
Cz = flat_projection(Cx)

Y ahora visualizamos los datos sobre el mapa.

In [ ]:
def plot_map(Z, Cz, centroids=None, labels=None, show_plot=False, make_figure=True):
    """
    Dibuja un mapa de Colombia y las ciudades, y opcionalmente los centroides

    Parameters
    ----------
    Z : numpy array
        Array con las coordenadas de las ciudades.
    Cz : numpy array
        Array con las coordenadas proyectadas de la frontera de Colombia.
    centroids : numpy array, optional
        Array con las coordenadas de los centroides.
        El valor por defecto es None (en cuyo caso no se dibujan centroides).
    show_plot : bool, optional
        Indica si se debe mostrar el gráfico. El valor por defecto es False.
    """

    # Create figure
    if make_figure:
        fig = plt.figure(figsize=(8, 8))
    # Plot Colombia
    plt.plot(Cz[:, 0], Cz[:, 1], c='gray')
    # Plot cities.
    plt.scatter(Z[:, 0], Z[:, 1], c=labels, marker='.', s=0.1)   # , s=ms)   #, c='cyan')




    if centroids is not None:
        plt.scatter(centroids[:, 0], centroids[:, 1], c= "red", marker='x',
                    label="Centroids")

    plt.axis('equal')
    plt.axis('off')

    if show_plot:
        plt.show()

    return

# Plot Colombia and locations
plot_map(Z, Cz, centroids=None, show_plot=True)

## 3. Algoritmos de agrupamiento (clustering)

### 3.1 K-means



K-means mostrando coordenadas de clusters y nº clientes


In [ ]:
# KMeans algorithm
kmeans = KMeans(n_clusters=num_centros, n_init='auto', random_state=0)
kmeans.fit(Z, sample_weight=demanda_x2)

# Obtiene el cluster asociado a cada muestra
labels = kmeans.labels_

# Obtiene los centroides
centroids = kmeans.cluster_centers_

# Obtiene el número total de puntos en cada cluster
num_points_in_cluster = np.bincount(labels)

# Score
score = kmeans.inertia_

# Dimensiona la figura
plot_map(Z, Cz, centroids=centroids, labels=labels, show_plot=False)
# Muestra el número total de puntos en cada cluster
for i in range(num_centros):
    plt.annotate(f'Centro [{i}]', (centroids[i, 0] + 30, centroids[i, 1]))

plt.title("KMeans Clustering")
plt.legend()
plt.show()

Estas son las coordenadas geográficas de los centroides:

In [ ]:
print("-- Coordenadas geográficas de los centroides:")
centroids_x = inverse_flat_projection(centroids)
lat_centroids = centroids_x[:, 0]
lon_centroids = centroids_x[:, 1]

for i in range(len(lat_centroids)):
    print(f'[{i}]: {lat_centroids[i]:.8f}, {lon_centroids[i]:.8f}')


In [ ]:
# Rango de valores de n_clusters que deseas probar (de 2 a 30)
range_n_clusters = range(2, 31)

# Inicializa una lista para almacenar las puntuaciones de inercia
inertia_scores = []

# Bucle para calcular la inercia para cada valor de n_clusters
for n_clusters in range_n_clusters:
    # Inicializa el agrupador KMeans con n_clusters y una semilla aleatoria
    kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=0)
    kmeans.fit(Z, sample_weight=demanda_x2)

    # Obtiene la inercia y la almacena en la lista
    inertia_scores.append(kmeans.inertia_)

    # Imprime la puntuación de inercia para cada n_clusters en una línea
    print(f"Para n_clusters={n_clusters}, la puntuación de inercia es: {kmeans.inertia_}")

# Visualización de las puntuaciones de inercia en dos gráficos. A la izquierda, en escala lineal, y a la derecha, en escala logarítmica
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range_n_clusters, inertia_scores, marker='o')
plt.title('Inercia vs número de clusters')
plt.xlabel('Número de clusters (n_clusters)')
plt.ylabel('Puntuación de Inercia')

plt.subplot(1, 2, 2)
plt.plot(range_n_clusters, inertia_scores, marker='o')
plt.yscale('log')
plt.title('Inercia vs número de clusters')
plt.xlabel('Número de clusters (n_clusters)')
plt.ylabel('Puntuación de Inercia (log)')

plt.show()

Para determinar el número de clusters, exploraremos diferentes opciones

### 3.1.2 K-medias con silhouette

In [ ]:
if compute_silouhette:

    # Supongamos que Z es tu matriz resultante

    # Rango de valores de n_clusters que deseas probar
    range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]

    # Inicializa cluster scores
    ch_score = []
    db_score = []

    # Creamos una figura para visualizar los resultados
    plt.figure(figsize=(8, 4))

    for n_clusters in range_n_clusters:

        # ###### Clustering #######################################################
        print(f"-- Aplicando k-means con {n_clusters} centroides      \r", end="")

        # Inicializa el agrupador KMeans con n_clusters y una semilla aleatoria
        clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)

        # Hacemos el fit sin usar los sample_weights, porque las metricas de
        # cluster no los utilizan
        cluster_labels = clusterer.fit_predict(Z)

        # Calcula el score de silueta promedio para todos los datos
        silhouette_avg = silhouette_score(Z, cluster_labels)

        ch_score.append(calinski_harabasz_score(Z, cluster_labels))
        db_score.append(davies_bouldin_score(Z, cluster_labels))

        # Calcula y visualiza los scores de silueta para cada muestra
        sample_silhouette_values = silhouette_samples(Z, cluster_labels)

        # ###### Visualización ####################################################

        # Inicializa una figura con 1 fila y 2 columnas
        fig, (ax1, ax2) = plt.subplots(1, 2)
        # fig.set_size_inches(18, 7)
        fig.set_size_inches(12, 4)

        # El 1er subplot es el plot de silueta
        ax1.set_xlim([-0.1, 1])
        # Inserta espacio en blanco entre plots de silueta individuales para demarcar claramente los clusters
        ax1.set_ylim([0, len(Z) + (n_clusters + 1) * 10])

        y_lower = 10
        for i in range(n_clusters):
            # Agrega scores de silueta para muestras pertenecientes al cluster i y ordénalos
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(
                np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values,
                facecolor=color, edgecolor=color, alpha=0.7)

            # Etiqueta los plots de silueta con números de cluster en el medio
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Calcula el nuevo y_lower para el próximo plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title(f"Silhouette plot for {n_clusters} clusters")
        ax1.set_xlabel("Silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # Línea vertical para el score de silueta promedio de todos los valores
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Limpia las etiquetas del eje y
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2do plot que muestra los clusters formados
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(Z[:, 0], Z[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors,
                    edgecolor="k")

        # Etiqueta los clusters
        centers = clusterer.cluster_centers_
        ax2.scatter(centers[:, 0], centers[:, 1], marker="o", c="white", alpha=1,
                    s=200, edgecolor="k")

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker=f"${i}$", alpha=1, s=50, edgecolor="k")

        ax2.set_title(f"Visualization of {n_clusters} clusters")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(f"Silhouette analysis for KMeans clustering with {n_clusters} clusters",
                    fontsize=14, fontweight="bold")

    plt.show()

In [ ]:
if compute_silouhette:
    # Rango de valores de n_clusters que deseas probar (de 2 a 50)
    range_n_clusters = range(2, 51)

    # Inicializa listas para almacenar puntuaciones de silueta
    silhouette_avg_list = []
    silhouette_values_list = []

    for n_clusters in range_n_clusters:
        print(f"-- Num clusters = {n_clusters}      \r", end="")

        # Inicializa el agrupador KMeans con n_clusters y una semilla aleatoria
        clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)
        # Hacemo el fit sin sample weights porque las metricas de cluster posteriores
        # no los utilizan
        cluster_labels = clusterer.fit_predict(Z)

        # Calcula el coeficiente de silueta promedio para todos los datos
        silhouette_avg = silhouette_score(Z, cluster_labels)
        silhouette_avg_list.append(silhouette_avg)

        # Calcula y almacena los valores de silueta para cada muestra
        silhouette_values = silhouette_samples(Z, cluster_labels)
        silhouette_values_list.append(silhouette_values)

    # Imprime el valor máximo del coeficiente de silueta y su correspondiente n_clusters
    max_silhouette_avg_index = np.argmax(silhouette_avg_list)
    print(f"The maximum silhouette score is {silhouette_avg_list[max_silhouette_avg_index]} for n_clusters = {range_n_clusters[max_silhouette_avg_index]}")

    # Visualización de las puntuaciones de silueta en un gráfico
    plt.figure(figsize=(15, 4))
    plt.plot(range_n_clusters, silhouette_avg_list, '.-')
    plt.ylabel("Silhouette score (higher is better)")
    plt.xlabel("Number of clusters")
    plt.show()

#### 4.1.3 K-means con Calinski y Davies Bouldin

In [ ]:
if compute_ckdb:
    # Rango de valores de n_clusters que deseas probar
    range_n_clusters = range(2, 102, 1)

    # Inicializa cluster scores
    ch_score = []
    db_score = []

    for n_clusters in range_n_clusters:
        print(f"-- Num clusters = {n_clusters}      \r", end="")

        # ###### Clustering #######################################################

        # Inicializa el agrupador KMeans con n_clusters y una semilla aleatoria
        clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)
        cluster_labels = clusterer.fit_predict(Z)

        ch_score.append(calinski_harabasz_score(Z, cluster_labels))
        db_score.append(davies_bouldin_score(Z, cluster_labels))

In [ ]:
if compute_ckdb:
    print(f"The Calinskk-Harabasz score is maximum for n_clusters = {range_n_clusters[np.argmax(ch_score)]}")
    print(f"The Davies-Bouldin score is maximum for n_clusters = {range_n_clusters[np.argmin(db_score)]}")

    plt.figure(figsize=(15, 4))
    plt.subplot(121)
    plt.plot(range_n_clusters, ch_score, '.-')
    plt.ylabel("Calinski-Harabasz score (higher is better)")
    plt.xlabel("Number of clusters")

    plt.subplot(122)
    plt.plot(range_n_clusters, db_score, '.-')
    plt.ylabel("Davies-Bouldin score (lower is better)")
    plt.xlabel("Number of clusters")
    plt.show()

### 3.2 K-medians

In [ ]:
if compute_kmedians:
    # KMedians algorithm
    K = num_centros

    # Selecciona centroides iniciales de forma aleatoria entre los datos
    initial_cluster_centers = Z[np.random.permutation(X.shape[0])[:K],:];

    # Aplica k-medians
    kmedians_instance = kmedians(Z, initial_cluster_centers)
    kmedians_instance.process();

    # Extrae clusters
    clusters = kmedians_instance.get_clusters()
    y_kmedians = np.zeros([X.shape[0]])
    y_kmedians[clusters[0]] = 0
    y_kmedians[clusters[1]] = 1
    C = np.array(kmedians_instance.get_medians())
    print(C)

In [ ]:
if compute_kmedians:
    # Plotting
    plot_map(Z, cx, cy, centroids=C, show_plot=False)
    plt.title("KMedians Clustering")
    plt.legend()
    plt.show()

    # crosstab(y, y_kmedians)

## 4. Matriz de Distancias

En general, es conveniente precalcular la matriz de distancias entre todas las ciudades antes de lanzar el cálculo de rutas.

Dicha matriz debería contener las distancias geodésicas. Sin embargo, esto consume mucho tiempo. Por este motivo, definimos tres funciones que realizan el cálculo con diferentes niveles de precisión y velocidad:

* **geodesic_distance**: es el más preciso (supone que la tierra es un elipsoide de revolución) pero es muy lento.
* **haversine_distance**: simplifica el cálculo suponiendo tierra exactamente esférica, y por tanto es algo más impreciso, pero bastante más rápido (~1min)
* **euclidean_distance**: hace una aproximacion de tierra plana, muy rápida pero bastante imprecisa, sobre todo en el cálculo de distancias paralelas al ecuador.

Es importante notar que `euclidean_distance` necesita como entrada la proyección de los datos sobre el plano calculada en la matriz ${\bf Z}$. El resto de funciones necesitan datos como latitud y longitud.

In [ ]:
# Initialize distance matrix
def geodesic_distance(X):
    """
    Calcula la matriz de distancias geodésicas entre las ciudades.
    Este método es el más exacto, pero es muy lento, y no se recomienda su uso
    para matrices de entrada con más de 100 ciudades.

    Parameters
    ----------
    X : numpy array
        Array con las coordenadas de las ciudades.

    Returns
    -------
    D : numpy array
        Matriz de distancias geodésicas entre las ciudades.
    """

    # Initialize distance matrix
    n_cities = X.shape[0]
    D = np.zeros((n_cities, n_cities))

    # Compute distance matrix
    for i in range(n_cities):
        print(f"-- City {i+1} out of {n_cities} \r", end="")
        for j in range(n_cities):
            if j > i:
                D[i][j] = geodesic(X[i], X[j]).kilometers
            else:
                D[i][j] = D[j][i]
    return D


def haversine_distance(X):
    """
    Calcula la matriz de distancias haversinas entre las ciudades.
    Este método es más rápido que el método geodésico, pero es menos exacto,
    porque supone que la Tierra es una esfera perfecta en lugar de un elipsoide.
    """

    # Radio de la Tierra en kilómetros
    R = 6371.0

    # Convertir de grados a radianes
    X_rad = np.radians(X)

    # Extraer latitudes y longitudes
    lat = np.array([X_rad[:, 0]]).T
    lon = np.array([X_rad[:, 1]]).T

    # Calcular las diferencias de latitudes y longitudes
    print("-- Calculando diferencias de latitudes y longitudes...")
    delta_lat = lat.T - lat
    delta_lon = lon.T - lon

    # Aplicar la fórmula del semiverseno
    print("-- Aplicando la fórmula del semiverseno...")
    cos_lat = np.cos(lat)
    a = (np.sin(delta_lat / 2.0)**2
         + cos_lat * cos_lat.T * np.sin(delta_lon / 2.0)**2)
    D = 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return D

def euclidean_distance(X):
    """
    Computes the euclidean distance matrix between cities. It is very fast, but
    also very inaccurate, because it assumes a flat Earth...
    """

    Z = flat_projection(X)

    S = Z @ Z.T
    modZ = np.array([np.diag(S)])
    D = np.sqrt(modZ + modZ.T - 2 * S)

    return D

def calcula_distancias(X, metric):
    """
    Calcula la matriz de distancias entre las ciudades, utilizando la métrica
    especificada.

    Parameters
    ----------
    X : numpy array
        Array con las coordenadas de las ciudades.
    metric : string
        Métrica de distancia a utilizar. Puede ser "euclidea", "haversine" o
        "geodesic".

    Returns
    -------
    D : numpy array
        Matriz de distancias entre las ciudades.
    """

    if metric == "euclidea":
        D = euclidean_distance(X)
    elif metric == "haversine":
        D = haversine_distance(X)
    elif metric == "geodesic":
        D = geodesic_distance(X)
    else:
        raise ValueError(f"Unknown distance metric: {metric}")

    return D

## 5. Planificación de rutas

### 5.1 Ruta simple

A modo de ensayo, para generar la ruta más corta que recorre todos los nodos y retorna al punto de partida. Este es un problema clásico en optimización combinatoria, denominado el _problema del viajante_ o TSP (Traveling Salesman Problem).

Para calcular la rota óptima utilizaremos la librería `ortools`.

Dado que el cálculo del camino más corto es computacionalmente costoso, permitimos el uso de un subconjunto de localizaciones.

Extraemos la submatriz con `n_loc_tsp` entradas, y añadimos una localización final de coordenadas nulas (que no tiene sentido como localización real, pero es un artificio útil para que el algoritmo de enrutamiento calcule una ruta de retorno).


In [ ]:
# Parámetros del algoritmo
num_routes = 1   # Solo una ruta (TSP clásico)

# Extrae submatriz de datos
Xsub = X[:n_loc_tsp, :]
# Añadimos punto virtual en el (0,0), que se fija como punto de partida
Xe = np.vstack((Xsub, np.array([[0, 0]])))

A continuación calculamos la matriz de distancias entre todas las localizaciones.

IMPORTANTE: la librería `ortools` requiere distancias enteras. Por este motivo, es necesario escalar la matriz de distancias por un factor entero grande, para garantizar cierta precisión. Un factor $1e6$ puede ser suficiente.

In [ ]:
# Calcula la matriz de distancias
D = calcula_distancias(Xe, distance_metric)

print(f"-- Maximum distance = {np.max(D)}")
print(f"-- Minimum distance = {np.min(D + 1e100*(D==0))}")
print(f"-- Distance matrix computed with dimension {D.shape}")

# Escalamos la matriz de distancias para convertirla en una matriz de enteros,
# que es el formato que necesita ortools
D = (scale_D * D).astype(int)
D[:, -1] = 10
D[-1, :] = 10

Para aplicar `ortools` es necesario definir dos funciones: una, que proporcione valores de la matriz de distancias, y otra que devuelva la solución obtenida para el TSP. Adicionalmente, definimos una función para calcular la longitud total de una ruta.

In [ ]:
def distance_callback(from_ind, to_index):
    """ Returns the distance between the two nodes. """
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_ind)
    to_node = manager.IndexToNode(to_index)

    return D[from_node][to_node]

def get_tsp_solution(manager, routing, solution):
    """
    Returns TSP solution.
    """

    index = routing.Start(0)
    route = [manager.IndexToNode(index)]

    while not routing.IsEnd(index):
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))

    return route

def tsp_score(D, route):
    """
    Computes the total score for the input route given by distance matrix D
    It assumes that the routes return to the starting location from the last
    city in the list

    Parameters
    ----------
    D : np.ndarray
        Distance matrix
    route : list of int
        List of indices of the route
    """

    # List of destination for all cities in route
    dest = route.copy()
    dest.append(dest.pop(0))

    return sum([D[o][d] for o, d in zip(route, dest)])

def plot_path(best_path, Z, Zsub, Cz):
    """
    Plots a route in the map using locations in X
    """

    plot_map(Z, Cz, show_plot=False)
    plt.plot(Zsub[best_path, 0], Zsub[best_path, 1], '-', c='green', linewidth=0.5)

Definimos los objetos que necesita el solver para funcionar:

In [ ]:
if compute_tsp:
    start = time()
    base_point = len(D) - 1   # len(D)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(n_loc_tsp + 1, num_routes, base_point)

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.first_solution_strategy = (
    #     routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Sugerencias de GPT para reducir la carga computacional
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS)
    # Cambiar PATH_CHEAPEST_ARC por alguno de estos:
    # SAVINGS, CHRISTOFIDES, ALL_UNPERFORMED

    # search_parameters.local_search_metaheuristic = (
    #     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

    # search_parameters.time_limit.seconds = 30  # Limita a 30 segundos, por ejemplo

    print(f"{time()-start:.4f} segundos")

Lanzamos el solver para encontrar la ruta:

In [ ]:
if compute_tsp:

    start = time()
    solution = routing.SolveWithParameters(search_parameters)
    print(f"{time()-start:.4f} segundos")

 FInalmente, capturamos la mejor ruta y la mostramos sobre el mapa.

In [ ]:
if compute_tsp:

    start = time()

    if solution:
        best_path = get_tsp_solution(manager, routing, solution)

    # Captura la secuencia de localizaciones recorrida por la ruta obtenica.
    best_path = [x for x in best_path[1:-1]]

    score_best_path = tsp_score(D / scale_D, best_path)

    print(f"-- Score = {score_best_path}")
    print(f"{time()-start:.4f} segundos")

    Z = flat_projection(X)
    Zsub = flat_projection(Xsub)
    plot_path(best_path, Z, Zsub, Cz)
    plt.plot(Zsub[best_path, 0], Zsub[best_path, 1], '-', c='green', linewidth=0.5)
    plt.show()

### 5.2. Vehicle Routing Problem (VRP)

Variables importantes:

* Z: coordenadas kilométricas de todas las tiendas.
* D: distancias entre todas las tiendas
* labels: lista del cluster asociado a cada tienda (labels[i] es el clustes de la tienda de coordenadas Z[i].
* centroids: Cordenadas de los centroides.
* num_points_in_cluster: tamaño de cada cluster.
* weights_x2: la demanda de cada tienda

Hacer todo el proceso para un solo cluster, por ejemplo, el i. Por ejemplo:

i = 3
...

Ahora nos interesa:

* El cluster i, tiene cooordenadas centroids[i].
* Las tiendas del centroide i: Z[k] es una tienda del cluster i si label[k]=i.

Proceso:
  1. Seleccionar de la matriz Z la submatriz Zi que contiene solamente las filas de Z que se corresponden con las posiciones en las que labels toma el valor i. Es decir, Z[k] estará en Zi si label[k]=i.

  2. Hacer lo mismo con weights_x2.

  3. Apilar las tiendas con su centroide: es decir, hacer la matriz extendida Zei que apile centroids[i] y Zi. Zei[0] será el centroide y debajo estarán todas las tiendas.

  4. Calcular la matriz Di de distancias entre todas las filas de la matriz extendida, Zei.

  5. Crear el data_model del CVRP:
````
data["distance_matrix"] = Di
data["demands"] = Los datos de las demandas de las tiendas correspondientes.
data["num_vehicles"] = el valor que quieras.
data["depot] = 0.  <-- la posición del centroide en la matriz Ze.
````

  6.  Ejecutar el código del CVRP.

  7. Si todo funciona para el cluster i, meter todo el código en un bucle.
````
    for i in range(7):
        # Todo el codigo aquí...
````        

Definimos, en primer lugar, un conjunto de funciones que nos permitan  procesar los datos con secuencias de comandos sencillas

In [ ]:
"""Capacited Vehicles Routing Problem (CVRP)."""
def create_data_model(Di, demands, vc=1_000):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = Di
    data["demands"] = demands
    # El número de vehículos tiene que soportar la demanda total
    data["num_vehicles"] = int((sum(demands) - 1) / vc) + 1
    print(f"-- Se necesitan {data['num_vehicles']} vehiculos")

    # Todos los vehiculos tienen la misma capacidad
    data["vehicle_capacities"] = [vc] * data["num_vehicles"]

    data["depot"] = 0
    return data


In [ ]:
def solve_cvrp(Di, demands, vc, tmax=100):
    """
    Solve the CVRP problem. This is the algorithmic core of this notebook.
    It solves the route planning problem for a single cluster.

    Parameters
    ----------
    Di : np.ndarray
        Distance matrix
    demands : array-like
        Demand of every location
    vc : int
        Capacity of every single vehicle.
    tmax : int, optional (default=100)
        Time limit for the computation of the routs from a single center
    """

    # Get distance
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    # Get demand.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    # Instantiate the data problem.
    data = create_data_model(Di, demands, vc=vc)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Register method to get distances
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Register method to get demands
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    # Add capacity constraints
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity")

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES)

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

    search_parameters.time_limit.FromSeconds(tmax)

    # Solve the problem.
    print("-- -- -- Running solver. This may take some time...")
    t0 = time()
    solution = routing.SolveWithParameters(search_parameters)
    print(f"-- -- CVRP solved in {time()-t0} secs.")

    return data, manager, routing, solution

In [ ]:
def print_solution(data, manager, routing, solution, scale=1e9):
    """Prints solution on console.
    """

    if not solution:
      print("-- No se ha encontrado ninguna solución")
      return

    print(f"-- Total distance: {solution.ObjectiveValue() / scale} km")
    total_distance = 0
    total_load = 0

    for vehicle_id in range(data["num_vehicles"]):
        print(f"-- -- Vehicle {vehicle_id}.")
        index = routing.Start(vehicle_id)

        plan_output = f"-- -- Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        nodes = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            nodes.append(node_index)

            try:
                route_load += data["demands"][node_index]
            except:
                print(f"-- -- Fallo en nodo {node_index}")
            plan_output += f" {node_index} Load({route_load}) ->"
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"-- -- Distance of the route: {route_distance /scale } km\n"
        plan_output += f"-- -- Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load

    print(f"-- Total distance of all routes: {total_distance/scale:.3f} km")
    print(f"-- Total load of all routes: {total_load}")

    return


#### 5.2.1. Análisis de una ruta simple

A continuacion analizamos una ruta individual


In [ ]:
def get_best_path(data, manager, routing, solution):
    """
    Extrae el conjunto de mejores rutas encontradas, una por cada vehiculo.

    Parameters
    ----------
    solution :
        Variable solución devuelta por el algoritmo de optimización

    Returns
    -------
        best_path : list of list
            Es una lista de listas. Cada lista es la secuencia de ciudades que
            debe recorrer un vehiculo
    """

    best_path = []
    path_size = []

    # Si no hay solución devuelve una lista vacia
    if not solution:
        return best_path, path_size

    # Obtener el mejor camino de la solución
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        best_path_i = []
        path_size_i = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            best_path_i.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            path_size_i += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)

        best_path.append(best_path_i)
        path_size.append(path_size_i)

    return best_path, path_size


In [ ]:
# Extraer submatriz Zi y apilar las tiendas con su centroide
# Supongamos que i es el índice del cluster que te interesa (en este caso, i=1)
i = cluster_de_prueba

# ###### Selecciona coordenadas y pesos del cluster i
# Filtrar las filas de Z correspondientes al cluster i
Xi = X[labels == i]

# Filtrar las demandas correspondientes al cluster i
demanda_total = demanda_media * n_cities
demanda_i = demanda_total * d_profile[labels == i]

# Elimina localizaciones sin demanda:
print(f"-- Dataset con {len(demanda_i)} localizaciones")
if remove_zero_demand_cities:
  n_zero = sum(demanda_i==0)
  if n_zero > 0:
      print(f"-- Hay {n_zero} localizaciones sin demanda")
      # Suprime todas las localizaciones con peso 0.
      Xi = Xi[demanda_i > 0]
      demanda_i = demanda_i[demanda_i > 0]
      print(f"-- Dataset reducido a {len(demanda_i)} localizaciones")

# Recorta dataset a un tamaño máximo (sólo  para depuración de código)
Xi = Xi[:nmax]
demanda_i = demanda_i[:nmax]

# Apilar el centroide y las tiendas
Xei = np.vstack([centroids_x[i], Xi])
demanda_i = np.hstack([[0], demanda_i])

# Calcula matriz de distancias
Di = calcula_distancias(Xei, distance_metric)

# Escala parametros a valores enteros.
# Esto es necesario porque ortools solamente admite valores enteros
demanda_i = (scale_M * demanda_i).astype(int)
vc_scaled = int(scale_M * vc)
Di = (scale_D * Di).astype(int)

plt.semilogy(sorted(demanda_i))
plt.xlabel("Cliente (ordenado por demanda)")
plt.ylabel("Demanda")
print(f"-- Demanda mínima: {min(demanda_i[1:])}")
print(f"-- Demanda mediana: {np.median(demanda_i[1:])}")
print(f"-- Demanda media: {np.mean(demanda_i[1:])}")
print(f"-- Demanda máxima: {max(demanda_i)}")
print(f"-- Demanda total: {sum(demanda_i)}")

Lanzamos el planificador de rutas:


In [ ]:
if compute_pruebavrp:

    print(f"-- Planificación de rutas para el cluster {i}")
    data, manager, routing, solution = solve_cvrp(Di, demanda_i, vc_scaled, tmax=tmax)

    # Print solution on console.
    print_solution(data, manager, routing, solution)
    best_path, path_size = get_best_path(data, manager, routing, solution)

In [ ]:
def plot_multipaths(best_path, Z, Zsub, Cz, centroids):
    """
    Plots a route in the map using locations in X
    """

    plot_map(Z, Cz, centroids=centroids, show_plot=False)

    if isinstance(best_path[0], list):
        for path_i in best_path:
           plt.plot(Zsub[path_i, 0], Zsub[path_i, 1], '-', linewidth=0.5)
    else:
        plt.plot(Zsub[best_path, 0], Zsub[best_path, 1], '-', c='green', linewidth=0.5)

    plt.show()

# Print solution on console.
if compute_pruebavrp and solution:

    # Llamar a la función de visualización
    Zei = flat_projection(Xei)
    plot_multipaths(best_path, Z, Zei, Cz, centroids)

#### 5.2.2. Planificación completa

In [ ]:
# Plot Colombia, the cities and the centroids
plot_map(Z, Cz, centroids=centroids, show_plot=False)

# Inicializa variables del bucle
distancia_total = 0
route_plan = []
route_plan_distances = []

demanda_total = demanda_media * n_cities
num_centros = centroids.shape[0]
data = [0] * num_centros
manager = [0] * num_centros
routing = [0] * num_centros
solution = [0] * num_centros

for i in range(centroids.shape[0]):

    print(f"\n----------------------------------------")
    print(f"-- Planificando rutas para el cluster {i}")

    # ###### Selecciona coordenadas y pesos del cluster i
    # Filtrar las filas de Z correspondientes al cluster i
    Xi = X[labels == i]

    # Filtrar las demandas correspondientes al cluster i
    demanda_i = demanda_total * d_profile[labels == i]

    # Elimina localizaciones sin demanda:
    print(f"-- -- Dataset con {len(demanda_i)} localizaciones")
    if remove_zero_demand_cities:
      n_zero = sum(demanda_i==0)
      if n_zero > 0:
          print(f"-- -- Hay {n_zero} localizaciones sin demanda")
          # Suprime todas las localizaciones con peso 0.
          Xi = Xi[demanda_i > 0]
          demanda_i = demanda_i[demanda_i > 0]
          print(f"-- -- Dataset reducido a {len(demanda_i)} localizaciones")

    # Recorta dataset a un tamaño máximo (sólo  para depuración de código)
    Xi = Xi[:nmax]
    demanda_i = demanda_i[:nmax]

    # Apilar el centroide y las tiendas
    Xei = np.vstack([centroids_x[i], Xi])
    demanda_i = np.hstack([[0], demanda_i])

    # Calcula matriz de distancias
    Di = calcula_distancias(Xei, distance_metric)

    # Escala parametros a valores enteros.
    # Esto es necesario porque ortools solamente admite valores enteros
    demanda_i = (scale_M * demanda_i).astype(int)
    vc_scaled = int(scale_M * vc)
    Di = (scale_D * Di).astype(int)

    print(f"-- -- Demanda mínima y máxima: {min(demanda_i[1:])} -- {max(demanda_i)}")
    print(f"-- -- Demanda total: {sum(demanda_i)}")

    data[i], manager[i], routing[i], solution[i] = solve_cvrp(Di, demanda_i, vc_scaled, tmax=tmax)
    print_solution(data[i], manager[i], routing[i], solution[i])

    # Obtiene las rutas calculadas para el cluster i
    best_path, path_size = get_best_path(data[i], manager[i], routing[i], solution[i])

    # Incluye el nodo de salida, que es también de retorno, al final de la ruta
    if isinstance(best_path[0], list):
        for path_k in best_path:
            path_k.append(path_k[0])
    else:
        best_path.append(best_path[0])

    # Añade esas rutas a una lista con todas las rutas
    route_plan.append(best_path)
    route_plan_distances.append(path_size)

    # Traza las rutas del cluster i
    if len(best_path) > 0:
        Zei = flat_projection(Xei)
        if isinstance(best_path[0], list):
            for path_k in best_path:
                plt.plot(Zei[path_k, 0], Zei[path_k, 1], '-', linewidth=0.5)
        else:
            plt.plot(Zei[best_path, 0], Zei[best_path, 1], '-', c='green', linewidth=0.5)

    # Distancia acumulada de todas las rutas y todos los clusters
    distancia_total += solution[i].ObjectiveValue() / scale_D

print(f"-- DISTANCIA TOTAL: {distancia_total:.3f} km")
plt.show()


In [ ]:
# Plot Colombia, the cities and the centroids
plt.figure(figsize=(16, 16))

for i in range(centroids.shape[0]):

    plt.subplot(4, 2, i+1)
    plot_map(Z, Cz, centroids=centroids, show_plot=False, make_figure=False)

    # ###### Selecciona coordenadas y pesos del cluster i
    # Filtrar las filas de Z correspondientes al cluster i
    Xi = X[labels == i]

    # Filtrar las demandas correspondientes al cluster i
    demanda_i = demanda_total * d_profile[labels == i]

    # Elimina localizaciones sin demanda:
    if remove_zero_demand_cities:
        print(f"-- -- Hay {sum(demanda_i==0)} localizaciones sin demanda")
        Xi = Xi[demanda_i > 0]

    # Recorta dataset a un tamaño máximo (sólo para depuración de código)
    Xi = Xi[:nmax]

    # Apilar el centroide y las tiendas
    Xei = np.vstack([centroids_x[i], Xi])

    # Obtiene las rutas calculadas para el cluster i
    best_path = route_plan[i]

    # Traza las rutas del cluster i
    xmin, xmax = 1e100, -1e100
    ymin, ymax = 1e100, -1e100

    if len(best_path) > 0:
        Zei = flat_projection(Xei)
        num_rutas = len(best_path)
        for path_k in best_path:
            plt.plot(Zei[path_k, 0], Zei[path_k, 1], '-', linewidth=0.5)
            xmin = min(xmin, min(Zei[path_k, 0]))
            xmax = max(xmax, max(Zei[path_k, 0]))
            ymin = min(ymin, min(Zei[path_k, 1]))
            ymax = max(ymax, max(Zei[path_k, 1]))

    d = 50
    xmin, xmax = xmin - d, xmax + d
    ymin, ymax = ymin - d, ymax + d

    plt.xlim([xmin, xmax])
    plt.ylim([ymin, ymax])
    plt.axis('on')
    plt.title(f"Centro {i}: {num_rutas} rutas")

print(f"-- DISTANCIA TOTAL: {distancia_total:.3f} km")
plt.show()

In [ ]:
print(route_plan)

In [ ]:
all_routes = []
for routes in route_plan_distances:
    all_routes += routes

media= distancia_total/len(all_routes)

plt.figure(figsize=(4,8))
plt.plot(np.array(all_routes) / scale_D)

#Agregar linea horizontal en la media
plt.axhline(y=media, color='r', linestyle='--')

plt.show()
print(f"La media es: {media}")